In [2]:
import pandas as pd
from datetime import datetime
import numpy as np
import json

In [3]:
#dataEvents = pd.read_csv('/Users/jruipere/Dropbox (MIT)/Game-based Assessment/data_processing_scripts/data/anonymized_dataset.csv', sep=";")

In [4]:
dataEvents = pd.read_csv('/Users/pedroantonio/Desktop/TFG/notebooks/anonamyze_all_data_collection.csv', sep=";")

In [5]:
levelsOfActivity = computeLevelsOfActivity(dataEvents)

In [6]:
levelsOfActivity

,group,user,task_id,metric,value
0,4fe25833f555e9903d2bb6bbeec3fbfb,0157bc278747755031338f18c5ba4233,1. One Box,timeTotal,62.018967
1,4fe25833f555e9903d2bb6bbeec3fbfb,0157bc278747755031338f18c5ba4233,2. Separated Boxes,timeTotal,110.271236
2,4fe25833f555e9903d2bb6bbeec3fbfb,0157bc278747755031338f18c5ba4233,3. Rotate a Pyramid,timeTotal,43.318789
3,4fe25833f555e9903d2bb6bbeec3fbfb,0157bc278747755031338f18c5ba4233,4. Match Silhouettes,timeTotal,821.921867
4,4fe25833f555e9903d2bb6bbeec3fbfb,0157bc278747755031338f18c5ba4233,5. Removing Objects,timeTotal,124.081586
...,...,...,...,...,...
74923,e6af7d42084352a39449e6d0a09b18cd,ffa946ca09448817699cabbb3dd78d56,Sugar Cones,ws-check_solution,1.000000
74924,e6af7d42084352a39449e6d0a09b18cd,ffa946ca09448817699cabbb3dd78d56,Tall and Small,ws-check_solution,1.000000
74925,e6af7d42084352a39449e6d0a09b18cd,ffa946ca09448817699cabbb3dd78d56,Unnecessary,ws-check_solution,3.000000
74926,e6af7d42084352a39449e6d0a09b18cd,ffa946ca09448817699cabbb3dd78d56,Warm Up,ws-check_solution,1.000000


In [1]:
def computeLevelsOfActivity(dataEvents, group = 'all'):

    
    dataEvents['time'] = pd.to_datetime(dataEvents['time'])
    dataEvents = dataEvents.sort_values('time')
    
    #iterates in the groups and users of the data
    dataEvents['group'] = [json.loads(x)['group'] if 'group' in json.loads(x).keys() else '' for x in dataEvents['data']]
    dataEvents['user'] = [json.loads(x)['user'] if 'user' in json.loads(x).keys() else '' for x in dataEvents['data']]
    dataEvents['task_id'] = [json.loads(x)['task_id'] if 'task_id' in json.loads(x).keys() else '' for x in dataEvents['data']]
    
    # removing those rows where we dont have a group and a user that is not guest
    dataEvents = dataEvents[((dataEvents['group'] != '') & (dataEvents['user'] != '') & (dataEvents['user'] != 'guest'))]
    dataEvents['group_user_id'] = dataEvents['group'] + '~' + dataEvents['user']
    dataEvents['group_user_task_id'] = dataEvents['group'] + '~' + dataEvents['user']+'~'+dataEvents['task_id']

         
    # filtering to only take the group passed as argument
    if(group != 'all'):
        dataEvents = dataEvents[dataEvents['group'].isin(group)]
          
    # the data is grouped by the necessary variables      
    activity_by_user = dataEvents.groupby(['group_user_id','group', 'user','group_user_task_id','task_id']).agg({'id':'count',
                                             'type':'nunique'}).reset_index().rename(columns={'id':'events',
                                                                                              'type':'different_events'}) 
    
    #indicate the index variable                                                                                                                                                               
    activity_by_user.index = activity_by_user['group_user_task_id'].values
    
    typeEvents = ['ws-snapshot','ws-paint', 'ws-rotate_view','ws-move_shape','ws-rotate_shape' ,'ws-scale_shape','ws-create_shape','ws-delete_shape','ws-undo_action','ws-redo_action', 'ws-check_solution']
    
   
        
    #initialize the metrics  
    activity_by_user['timeTotal'] = np.nan
    activity_by_user['inactive_time'] = np.nan
    activity_by_user['event'] = np.nan
    activity_by_user['different_events'] = np.nan
    activity_by_user['active_time'] = np.nan
    for event in typeEvents:
        activity_by_user[event] = 0
    
    #initialize the data structures
    userFunnelDict = dict()  
    puzzleEvents = dict()
    eventsDiff = []
    eventsDiff_puzzle = dict()
    timePuzzle = dict()
    globalTypesEvents = dict()
    eventInitial = dict()
    totalTime = dict()
    
      
    for user in dataEvents['group_user_id'].unique():
        
        # Computing active time
        previousEvent = None
        theresHoldActivity = 60 # np.percentile(allDifferences, 98) is 10 seconds
        limit = 3600
        activeTime = []
        
        user_events = dataEvents[dataEvents['group_user_id'] == user]
        user_puzzle_key = None

        for enum, event in user_events.iterrows():
            
            # If it is the first event
            if(previousEvent is None):
                previousEvent = event
                continue
            
            if(event['type'] in ['ws-start_level', 'ws-puzzle_started']):
                
                #create id: group+user+task_id                                                                              
                user_puzzle_key = event['group'] + '~' + event['user'] + '~' + json.loads(event['data'])['task_id']
                
                if(user_puzzle_key not in totalTime.keys()):
                    totalTime[user_puzzle_key] = 0
                    
                delta_seconds = (event['time'] - previousEvent['time']).total_seconds()
                if((delta_seconds < limit)):
                    totalTime[user_puzzle_key] += delta_seconds
                previousEvent = event
                
                       
                # initialize if the id is new                                                                              
                if(user_puzzle_key not in puzzleEvents.keys()):
                    totalTime[user_puzzle_key] = 0
                    puzzleEvents[user_puzzle_key]= 1
                    eventsDiff_puzzle[user_puzzle_key] = []
                    eventsDiff_puzzle[user_puzzle_key].append(event['type'])
                    timePuzzle[user_puzzle_key] = 0
                    globalTypesEvents[user_puzzle_key] = dict()
                    for ev in typeEvents:
                        globalTypesEvents[user_puzzle_key][ev]= 0
                        
                       
            
            # the event is not final event
            if(event['type'] not in ['ws-exit_to_menu', 'ws-puzzle_complete', 'ws-create_user', 'ws-login_user']): 
                    puzzleEvents[user_puzzle_key] += 1
                                                                                              
                    #add the event type                                                                          
                    eventsDiff_puzzle[user_puzzle_key].append(event['type'])
                    
                    #calculate the duration of the event                                                                          
                    delta_seconds = (event['time'] - previousEvent['time']).total_seconds()
                    if((delta_seconds < limit)):
                        totalTime[user_puzzle_key] += delta_seconds
                    if((delta_seconds < theresHoldActivity)):
                        timePuzzle[user_puzzle_key] += delta_seconds

                    previousEvent = event 
                    
                    #update event counters by type
                    if(event['type'] in typeEvents):
                        globalTypesEvents[user_puzzle_key][event['type']] +=1
                    
                        
            # the puzzle ends        
            if(event['type'] in ['ws-exit_to_menu', 'ws-puzzle_complete']):
                
                    puzzleEvents[user_puzzle_key] += 1
                    
                    #add the event type                                                                         
                    eventsDiff_puzzle[user_puzzle_key].append(event['type'])
                    
                    #calculate the duration of the event                                                                          
                    delta_seconds = (event['time'] - previousEvent['time']).total_seconds()
                    if((delta_seconds < limit)):
                        totalTime[user_puzzle_key] += delta_seconds
                    if((delta_seconds < theresHoldActivity)):
                        timePuzzle[user_puzzle_key] += delta_seconds
                        

                    previousEvent = event
    
    # add the data by group_user_task_id            
    for i in dataEvents['group_user_task_id'].unique():
        key_split = i.split('~')
        if(key_split[2] != ''):
            activity_by_user.at[i, 'timeTotal'] = totalTime[i]
            activity_by_user.at[i, 'inactive_time'] = totalTime[i]-timePuzzle[i]
            activity_by_user.at[i, 'event'] = puzzleEvents[i]
            activity_by_user.at[i, 'different_events'] = len(set(eventsDiff_puzzle[i]))
            activity_by_user.at[i, 'active_time'] = timePuzzle[i]
            for event in typeEvents:
                activity_by_user.at[i, event] = globalTypesEvents[i][event]

    #delete row with NaN
    activity_by_user.dropna(inplace=True)
    #delete group_user_task_id column
    activity_by_user.drop(columns=['group_user_task_id'], inplace=True)
    
    #data output preparation                                                                                          
    activity_by_user = pd.melt(activity_by_user, id_vars=['group', 'user','task_id'], 
        value_vars=['timeTotal','inactive_time','event','different_events', 'active_time','ws-snapshot','ws-paint','ws-rotate_view','ws-rotate_shape','ws-move_shape','ws-scale_shape','ws-create_shape','ws-delete_shape','ws-undo_action','ws-redo_action','ws-check_solution'], 
        var_name='metric', value_name='value')
        
    return activity_by_user